In [1]:
import pyspark
from pyspark.sql import SparkSession,DataFrame
import requests
import json 
from io import BytesIO
import pandas as pd
import os
import sys

In [2]:
import pyspark
print(pyspark.__version__)


3.5.6


In [3]:
def create_context() -> SparkSession:

    # Usa el mismo intérprete que el kernel del notebook
    os.environ["PYSPARK_PYTHON"] = sys.executable
    os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
    spark = SparkSession.builder\
        .appName("IcebergWritedata") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
        .config("spark.sql.catalog.spark_catalog.type", "hadoop") \
        .config("spark.sql.catalog.spark_catalog.warehouse", "../data/warehouse") \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .getOrCreate()
    #.config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.3") \
    return spark

In [4]:
def get_api_endpoint_excel(spark:SparkSession,path:str,filter:str = None) -> DataFrame:

    if filter:
        response = requests.get(f"{path}?{filter}")
    else:
        response = requests.get(f"{path}")
    if response.status_code == 200:
        # Leer el archivo Excel con pandas desde memoria
        excel_file = BytesIO(response.content)
        df_pandas = pd.read_excel(excel_file)

        # Convertir el DataFrame de pandas a Spark
        df_spark = spark.createDataFrame(df_pandas)

        # Mostrar los primeros registros
        return df_spark
    else:
        print(f"Error {response.status_code}: no se pudo obtener el archivo.")


In [5]:
path = "https://dataestur.azure-api.net/API-SEGITTUR-v1/AENA_DESTINOS_DL"
filter = "desde%20%28a%C3%B1o%29=2004&Aeropuerto%20AENA=JT%20Barcelona-El%20Prat"

spark = create_context()
df = get_api_endpoint_excel(spark,path,filter)

df.show()


25/06/11 19:25:05 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 10.192.39.199 instead (on interface wlo1)
25/06/11 19:25:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 19:25:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)



In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS spark_catalog.local_db")
# Guardar tabla Iceberg
df.writeTo("spark_catalog.local_db.aena_barcelona").using("iceberg").createOrReplace()

In [ ]:
path = "https://dataestur.azure-api.net/API-SEGITTUR-v1/EOH_PUNT_TUR_DL"
filter = "Punto%20tur%C3%ADstico=Todos&Lugar%20de%20residencia=Todos&Provincia=Todos"

df = get_api_endpoint_excel(spark,path,filter)

In [ ]:
# Guardar tabla Iceberg
df.writeTo("spark_catalog.local_db.hoteles_punto_turistico").using("iceberg").createOrReplace()